In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05132102966308594
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 12.341000067336715
200000 26.516076762820017
300000 25.234961585733775
400000 29.26921696922721
500000 25.76015681847024
600000 29.107433464452072
700000 37.01554446400315
800000 39.48569329102859
900000 45.195112253374255
1000000 47.179060297609205
1100000 46.753729631750964
1200000 33.89533537721979
1300000 24.73860890121901
1400000 45.629407894933586
1500000 45.78717286357799
1600000 46.26993493326595
1700000 46.997675209743456
1800000 49.2694037461931
1900000 48.35230546425568
2000000 46.61341274877836
2100000 46.59611879166876
2200000 46.72218113996008
2300000 46.951765724416774
2400000 38.562774018012846
2500000 19.619866811181467
2600000 46.0140882499886
2700000 46.068681888016776
2800000 45.728664640353024
2900000 45.29290205467416
3000000 45.78129

29800000 46.302629737818556
29900000 45.81477489119042
30000000 45.80815666876666
30100000 23.59866261736945
30200000 35.9146293681826
30300000 44.54219377213191
30400000 45.57184256306873
30500000 46.784769012818394
30600000 46.79950786847139
30700000 45.80383127249218
30800000 47.31268964576882
30900000 48.04643963739279
31000000 48.15654453756361
31100000 23.230476008927802
31200000 44.368342027295
31300000 49.663195126717746
31400000 50.78908130773391
31500000 43.27504706542694
31600000 26.004001914458126
31700000 49.91978186246256
31800000 50.16747578490715
31900000 51.570080314747365
32000000 50.48231860127489
32100000 50.72498647477542
32200000 48.479667526513104
32300000 47.65742936593389
32400000 45.88865785396794
32500000 25.417004696859
32600000 33.6271279053919
32700000 45.233361819421454
32800000 47.547374675035535
32900000 47.78795359511016
33000000 49.67619337735948
33100000 47.65506412541113
33200000 50.15409968492302
33300000 50.133720565042665
33400000 50.677828742310

60000000 45.7567231878064
60100000 45.42923190787258
60200000 47.249605903212064
60300000 48.3639610359523
60400000 48.93960659540748
60500000 49.602407688890416
60600000 37.17593963251709
60700000 28.54371016518058
60800000 46.43064098014634
60900000 44.96869921038663
61000000 45.42155529352876
61100000 44.162861304168636
61200000 43.58968107804216
61300000 41.58049743892635
61400000 13.827300791883657
61500000 40.78120863425563
61600000 42.922248649307555
61700000 44.390345748125476
61800000 43.190784435353244
61900000 46.298736816836296
62000000 44.51000182376805
62100000 38.007478790676956
62200000 29.189837947378944
62300000 48.76183665487379
62400000 47.72540976739891
62500000 47.522835192774
62600000 47.68656223214947
62700000 46.6464114865206
62800000 45.90091160524312
62900000 47.06790171301724
63000000 44.62514679728825
63100000 18.880628760569937
63200000 43.978619897808436
63300000 43.28149672019914
63400000 42.39387436320562
63500000 40.38335308930988
63600000 40.287583896

90100000 39.20290825208583
90200000 39.446055606615666
90300000 12.824451470473758
90400000 40.58670371006848
90500000 41.03869932599929
90600000 38.52607701266158
90700000 39.63868483731415
90800000 21.378057642469187
90900000 35.92900382880248
91000000 43.226605350015035
91100000 41.91632732904473
91200000 44.938203841388834
91300000 43.96620514905028
91400000 42.63965856356449
91500000 43.41130910955477
91600000 43.679739640541776
91700000 41.942367098555415
91800000 42.62968160029902
91900000 40.90192814310559
92000000 41.30781072310569
92100000 33.66253232163221
92200000 19.165615483428585
92300000 37.784303836034816
92400000 37.09952061407999
92500000 37.86596322329566
92600000 39.54809688254279
92700000 9.404610803979033
92800000 40.79153582464178
92900000 37.444069747517744
93000000 38.63084058949246
93100000 36.39007197832154
93200000 17.613211386607844
93300000 37.28082337767606
93400000 40.63144612288745
93500000 39.63634666537732
93600000 41.42941683599424
93700000 40.69076

119400000 36.19878390786734
119500000 19.782711914803333
119600000 37.60983174614552
119700000 38.71260814109982
119800000 36.56132825284652
119900000 39.089373892905165
120000000 40.76801189792951
120100000 42.06958262244996
120200000 42.65293744801324
120300000 42.356437691950674
120400000 39.37744393778598
120500000 40.74507294329261
120600000 39.24781901276979
120700000 38.53581971323038
120800000 37.196270564605605
120900000 38.03078547307263
121000000 35.5865070301137
121100000 36.28869777583237
121200000 27.928529577401225
121300000 22.25547185581307
121400000 34.91907001200109
121500000 27.74499787025325
121600000 19.173697182615733
121700000 34.50851686257807
121800000 31.29502205441401
121900000 20.868750619222695
122000000 36.401905345315136
122100000 36.675345181037216
122200000 35.46102770092342
122300000 37.25827832248327
122400000 37.31759495172295
122500000 39.60624085848043
122600000 39.36695297676743
122700000 39.48633754203237
122800000 40.835057787344276
122900000 4

...

494400000 6.831887442327206
494500000 5.135079522278701
494600000 6.156104759707785
494700000 7.784057385983431
494800000 7.636375508289401
494900000 7.8311527292618415
495000000 7.93021111659282
495100000 3.563979253743816
495200000 8.14983166286839
495300000 7.726551596860039
495400000 7.589791301778318
495500000 7.368389480549223
495600000 6.27848662719241
495700000 6.4168440199061125
495800000 5.601573961827294
495900000 7.759335178855203
496000000 7.412269981580748
496100000 7.776002527020634
496200000 7.618508773766038
496300000 3.8185374526260643
496400000 7.66722652783429
496500000 7.832225791673932
496600000 7.360096373942718
496700000 7.366797245358898
496800000 5.530471900008999
496900000 6.798563965286018
497000000 5.617789035215136
497100000 8.01225911734069
497200000 7.28436336605885
497300000 7.60140065508576
497400000 7.453040240497163
497500000 3.9254255434803147
497600000 7.145494779575977
497700000 7.76366031367397
497800000 7.0954568318658575
497900000 7.17177233768

523600000 5.899818962865735
523700000 6.402726389191261
523800000 6.052996133138909
523900000 4.442050372455173
524000000 2.9302992997196418
524100000 4.0352846412126135
524200000 5.4260599638269715
524300000 5.7447648221185865
524400000 6.33101513147453
524500000 5.884519324692325
524600000 5.680635634661241
524700000 6.247305961711607
524800000 5.479300082822269
524900000 5.250963967634911
525000000 6.1631278506449725
525100000 4.555653003204825
525200000 2.904816635500199
525300000 3.5860644281550385
525400000 5.289662662700907
525500000 5.8450773657692725
525600000 5.383406953941813
525700000 5.442270316345721
525800000 6.7675548207282965
525900000 6.027391089288562
526000000 5.754778025033441
526100000 6.975143856643028
526200000 5.664076977104595
526300000 4.528105418582688
526400000 2.0276056525417903
526500000 4.819322815413118
526600000 5.706753833047469
526700000 5.413721766289451
526800000 6.459088665710062
526900000 5.669120768711596
527000000 6.6313632881962
527100000 7.58

552600000 3.820356874804736
552700000 4.1366981934357225
552800000 3.66938889169263
552900000 2.7248498564535484
553000000 4.163159383844474
553100000 4.366147208187323
553200000 3.099914693891304
553300000 4.457079392364091
553400000 4.613847742994148
553500000 4.721716978250792
553600000 4.3794411722859925
553700000 2.871013660537066
553800000 4.190027687882603
553900000 4.048488425015752
554000000 3.716738579796243
554100000 2.593557658601918
554200000 4.050139425165405
554300000 4.14444605225896
554400000 3.2168750725350774
554500000 4.410588966738516
554600000 4.502079619946581
554700000 4.5260822962429295
554800000 4.50589435052783
554900000 2.5280632381224453
555000000 4.411059060308481
555100000 4.086313260795732
555200000 3.6822809998261787
555300000 2.415557577352478
555400000 4.012959448966814
555500000 4.04660741885064
555600000 2.991775560974927
555700000 4.545957469381254
555800000 4.471434479703569
555900000 4.398578750142275
556000000 4.5775082152694315
556100000 2.6052

580200000 1.968265483304186
580300000 1.8095007217359191
580400000 1.9260703299313904
580500000 2.103978359233151
580600000 2.7413291659250514
580700000 2.735799106136012
580800000 1.8209768215182964
580900000 2.0218090290962087
581000000 2.0618924700786114
581100000 1.9353778961490717
581200000 2.7629741911421175
581300000 2.7060893137943944
581400000 1.795616864273625
581500000 1.9032003442428622
581600000 2.087827271171093
581700000 1.933961480041356
581800000 2.8599020188691
581900000 2.8458643815057094
582000000 1.5264509829923392
582100000 2.0747220965568616
582200000 1.8646457373160163
582300000 1.8645382452998511
582400000 2.4111868064936495
582500000 2.533059795761053
582600000 1.5027254450984233
582700000 1.7852019563618453
582800000 1.6275443669876217
582900000 1.86845556062762
583000000 2.7471719864003097
583100000 1.8984935268588503
583200000 1.4437167160135043
583300000 2.0434036522893666
583400000 1.792005745961396
583500000 1.421358332619663
583600000 2.211178646085058


608700000 1.1918405334949804
608800000 0.9953919214453657
608900000 0.9514787579110264
609000000 0.8688879351721605
609100000 1.0236341534164013
609200000 0.6229270170387408
609300000 0.8487501055756714
609400000 1.0088257467961166
609500000 0.9168497135969321
609600000 0.9508824936717529
609700000 0.839868880095776
609800000 0.7850105109105647
609900000 1.1321144301067618
610000000 1.268876660226606
610100000 0.821436691144367
610200000 0.914606338097365
610300000 1.040001637923877
610400000 0.6981085117460907
610500000 1.2520385746446245
610600000 0.8309748615690874
610700000 0.6690696715214669
610800000 0.9104251103933421
610900000 0.8711102563973062
611000000 0.4194611401362439
611100000 0.9662563358783893
611200000 0.9923624872264565
611300000 0.5251819088066438
611400000 0.6721763162259929
611500000 1.1913395208110802
611600000 0.76949305715003
611700000 0.8447049157516904
611800000 1.1172179056447578
611900000 0.61002344712315
612000000 1.0972016951662997
612100000 1.09045205811